 # Get started with TensorFlow 2.0
 In this example we train an image classifier.

 **Load packages**

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.python.client import device_lib
import datetime

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



 **Print versions**

In [3]:
!cat /etc/*release

DISTRIB_ID=LinuxMint
DISTRIB_RELEASE=19.2
DISTRIB_CODENAME=tina
DISTRIB_DESCRIPTION="Linux Mint 19.2 Tina"
NAME="Linux Mint"
VERSION="19.2 (Tina)"
ID=linuxmint
ID_LIKE=ubuntu
PRETTY_NAME="Linux Mint 19.2"
VERSION_ID="19.2"
HOME_URL="https://www.linuxmint.com/"
SUPPORT_URL="https://forums.ubuntu.com/"
BUG_REPORT_URL="http://linuxmint-troubleshooting-guide.readthedocs.io/en/latest/"
PRIVACY_POLICY_URL="https://www.linuxmint.com/"
VERSION_CODENAME=tina
UBUNTU_CODENAME=bionic
cat: /etc/upstream-release: Is a directory


In [4]:
!nvidia-smi --query-gpu=driver_version --format=csv,noheader

430.26


In [5]:
!cat /usr/local/cuda*/version.txt

CUDA Version 10.0.130


In [6]:
!cat /usr/include/x86_64-linux-gnu/cudnn_v*.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 2
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


In [7]:
!dpkg -l | grep TensorRT

ii  graphsurgeon-tf                                             5.0.2-1+cuda10.0                                amd64        GraphSurgeon for TensorRT package
ii  libnvinfer-dev                                              5.1.5-1+cuda10.0                                amd64        TensorRT development libraries and headers
ii  libnvinfer5                                                 5.1.5-1+cuda10.0                                amd64        TensorRT runtime libraries
ii  uff-converter-tf                                            5.0.2-1+cuda10.0                                amd64        UFF converter for TensorRT package


In [8]:
print(tf.__version__)




2.0.0-rc0


 **Check GPU**

In [9]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13947997316737064487, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8697043822975493961
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7769106307604515159
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1487339520
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7261852161548268761
 physical_device_desc: "device: 0, name: GeForce MX150, pci bus id: 0000:02:00.0, compute capability: 6.1"]

In [10]:
tf.test.is_gpu_available()

True

In [11]:
tf.test.gpu_device_name()



'/device:GPU:0'

 **Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers**

In [12]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
x_train, x_test = x_train / 255.0, x_test / 255.0
print("Training set shape: " + str(x_train.shape))
print("Test set shape: " + str(x_test.shape))



Training set shape: (60000, 28, 28, 1)
Test set shape: (10000, 28, 28, 1)


 **Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training**

In [13]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)), 
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), 
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(10, activation='softmax')
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                1

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



 **Train and evaluate the model**

In [16]:
tms_start = datetime.datetime.now()
model.fit(x_train, y_train, epochs=30, batch_size=64)
tms_end = datetime.datetime.now()
tms_delta = tms_end - tms_start
print("Elapsed: " + str(tms_delta))
print("Elapsed in seconds: " + str(int(tms_delta.total_seconds())))

Train on 60000 samples
Epoch 1/30
60000/60000 [==============================] - 11s 179us/sample - loss: 0.4017 - accuracy: 0.8954
Epoch 2/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.1150 - accuracy: 0.9685
Epoch 3/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0835 - accuracy: 0.9763
Epoch 4/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0677 - accuracy: 0.9805
Epoch 5/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0579 - accuracy: 0.9835
Epoch 6/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0512 - accuracy: 0.9849
Epoch 7/30
60000/60000 [==============================] - 8s 129us/sample - loss: 0.0493 - accuracy: 0.9857
Epoch 8/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0417 - accuracy: 0.9878
Epoch 9/30
60000/60000 [==============================] - 8s 130us/sample - loss: 0.0395 - accuracy: 0.9881
Epoc

In [17]:
model.evaluate(x_test, y_test, verbose=0)



[0.03880422692706253, 0.9916]